<b>Importing Libraries</b>

In [1]:
import torch
import random
import json
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer


<b>Intents Data File</b>

In [2]:
ourData = {"intents": [
    {
      "tag": "greeting",
      "patterns": [
        "Hi",
        "Hey",
        "How are you",
        "Is anyone there?",
        "Hello",
        "Good day"
      ],
      "responses": [
        "Hey :-)",
        "Hello, thanks for visiting",
        "Hi there, what can I do for you?",
        "Hi there, how can I help?"
      ]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "See you later", "Goodbye"],
      "responses": [
        "See you later, thanks for visiting",
        "Have a nice day",
        "Bye! Come back again soon."
      ]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "That's helpful", "Thank's a lot!"],
      "responses": ["Happy to help!", "Any time!", "My pleasure"]
    },
    {
      "tag": "items",
      "patterns": [
        "Which items do you have?",
        "What kinds of items are there?",
        "What do you sell?"
      ],
      "responses": [
        "We sell athletic greens powder",
        "We have athletic greens powder"
      ]
    },
    {
      "tag": "payments",
      "patterns": [
        "Do you take credit cards?",
        "Do you accept Mastercard?",
        "Can I pay with Paypal?",
        "Are you cash only?"
      ],
      "responses": [
        "We accept VISA, Mastercard and Paypal",
        "We accept most major credit cards, and Paypal"
      ]
    },
    {
      "tag": "delivery",
      "patterns": [
        "How long does delivery take?",
        "How long does shipping take?",
        "When do I get my delivery?"
      ],
      "responses": [
        "Delivery takes 2-4 days",
        "Shipping takes 2-4 days"
      ]
    },
    {
        "tag": "diet",
        "patterns": [
            "Is it diet friendly?",
            "Is it Keto?",
            "Is it Vegan?",
            "Is it Low Carb?"
        ],
        "responses": [
            "Product is Diet Friendly,it is Vegetarian, Vegan, Paleo, Keto and Low-Carb"
        ]
    },
    {
        "tag": "GMO",
        "patterns": [
            "Does it have GMO's?",
            "Does it have Dairy or Nuts?",
            "Is it for Lactose Intolerant?"
        ],
        "responses": [
            "It has No GMO's, No Herbicides or Pesticides, No Artificial Colors/Flavors, Preservatives or Sweeteners, No Gluten, No Dairy, No Corn, No Egg, No Peanuts, No Animal Byproducts, No Lactose, Sucrose or Dextrose"
        ]
    },
    {
      "tag": "source",
      "patterns": [
        "Where is your product made?",
        "Where is your powder sourced from?",
        "Is it made in US?"
      ],
      "responses": [
        "It is made in New Zealand and it is NSF certified for sports."
      ]
    },
    {
      "tag": "certified",
      "patterns": [
        "Is it certified?",
        "Is it FDA approved?"
      ],
      "responses": [
        "It is NSF certified for sports."
      ]
    },
    {
      "tag": "price",
      "patterns": [
        "Cost",
        "Subscription",
        "What is the price?"
      ],
      "responses": [
        "One time purchase is $99 for 30 servings, for others please visit the 'Shop' page."
      ]
    }
  ]
}

<b>Defining Functions for Tokenization, Stemming and Bag of Words</b>

In [3]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    #for indexed and all words
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

<b>Creating classes for Neural Network Model</b>

In [4]:
#Feed Forward Neural Net
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes) #num_classes as ouput 
        self.relu = nn.ReLU() #activation function reLu
    
    def forward(self, x):        
        out = self.l1(x)
        out = self.relu(out)     #activation function which gets x out 
        out = self.l2(out)       #2nd linear layer out 
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

<b>Preparing tokenized, stemmed words and calling Neural Net Model</b>

In [5]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#from nltk_utils import bag_of_words, tokenize, stem
#from model import NeuralNet

#with open('intents.json', 'r') as f:
#    intents = json.load(f)
intents = ourData

all_words = []
tags = []
xy = []  #this will later on hold all the words and tags 
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair , tuple
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy: #unpack the tuple
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words) #calling the function bag of words
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot,  ( in tags indexed)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

#created class and inherit Dataset, implement init function
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    #later access dataset[idx], support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)
#loss function and the optimizer for gradient descent,loss function of the model which computes the 
#mean-squared error between the input and the target 
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

38 patterns
11 tags: ['GMO', 'certified', 'delivery', 'diet', 'goodbye', 'greeting', 'items', 'payments', 'price', 'source', 'thanks']
78 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'approv', 'are', 'bye', 'can', 'carb', 'card', 'cash', 'certifi', 'cost', 'credit', 'dairi', 'day', 'deliveri', 'diet', 'do', 'doe', 'fda', 'for', 'friendli', 'from', 'get', 'gmo', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'in', 'intoler', 'is', 'it', 'item', 'keto', 'kind', 'lactos', 'later', 'long', 'lot', 'low', 'made', 'mastercard', 'my', 'nut', 'of', 'onli', 'or', 'pay', 'paypal', 'powder', 'price', 'product', 'see', 'sell', 'ship', 'sourc', 'subscript', 'take', 'thank', 'that', 'the', 'there', 'us', 'vegan', 'what', 'when', 'where', 'which', 'with', 'you', 'your']
78 11
Epoch [100/1000], Loss: 1.4325
Epoch [200/1000], Loss: 0.1044
Epoch [300/1000], Loss: 0.0267
Epoch [400/1000], Loss: 0.0150
Epoch [500/1000], Loss: 0.0030
Epoch [600/1000], Loss: 0.0020
Epoch [700/100

<b>Implementing and Launching the ChatBot</b>

In [6]:
#import random
#import json

#import torch

#from modelchatprojectNLP import NeuralNet
#from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#with open('intents.json', 'r') as json_data:
#    intents = json.load(json_data)
intents = ourData
FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Team"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]
#applying softmax to get probabilities of the predictions, if prob is large enough > 0.75 then print response
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Sorry, I do not understand. Please call at 1-888-390-4029 or email at support@athleticgreens.com for further support.")

Let's chat! (type 'quit' to exit)
You: Hello
Team: Hello, thanks for visiting
You: Hi there 
Team: Hi there, how can I help?
You: What do you sell
Team: We sell athletic greens powder
You: Is it certified?
Team: It is NSF certified for sports.
You: Where is it made
Team: Sorry, I do not understand. Please call at 1-888-390-4029 or email at support@athleticgreens.com for further support.
You: Where is your product sourced from?
Team: It is made in New Zealand and it is NSF certified for sports.
You: Is it diet friendly?
Team: Product is Diet Friendly,it is Vegetarian, Vegan, Paleo, Keto and Low-Carb
You: Do you take credit cards?
Team: We accept most major credit cards, and Paypal
You: How long is shipping?
Team: One time purchase is $99 for 30 servings, for others please visit the 'Shop' page.
You: How long is delivery
Team: One time purchase is $99 for 30 servings, for others please visit the 'Shop' page.
You: Goodbye
Team: See you later, thanks for visiting
You: quit
